### This notebook explores the different features of pyproj library

In [25]:
from pyproj import CRS, database, aoi

In [37]:
def search_crs_by_area(area_name, object_type="all", filter_text=None):
    """
    Searches for CRS applicable to a given area.

    Parameters
    ----------
    area_name : str or pyproj.aoi.AreaOfInterest
        The name of the area or AreaOfInterest object to search for.
    object_type : str, optional
        The type of CRS or other database object to search for.
        Can be "all", "geographic", "projected", or other CRS types supported by pyproj.
        Defaults to "all".
    filter_text : str, optional
        A string of text to filter the results by, performing a case-insensitive
        search on the `area_of_use` attribute of each CRS. Defaults to None.

    Returns
    -------
    list
        A list of CRS objects that are applicable to the given area and CRS type.
    """

    if object_type == "all":
        crs_list = database.query_crs_info(area_of_interest=area_name, contains=False)
    else:
        crs_list = database.query_crs_info(
            area_of_interest=area_name, pj_types=object_type.upper(), contains=False
        )

    if filter_text:
        crs_list = [
            crs
            for crs in crs_list
            if filter_text.lower() in crs.area_of_use.name.lower()
        ]

    return crs_list


In [38]:
argentina_bbox = (
    -73.582520,
    -55.162286,
    -21.781349,
    -53.637451
)  # (minx, miny, maxx, maxy)

In [39]:
argentina_aoi = aoi.AreaOfInterest(
    west_lon_degree=argentina_bbox[0],
    south_lat_degree=argentina_bbox[3],
    east_lon_degree=argentina_bbox[1],
    north_lat_degree=argentina_bbox[2]
)


In [40]:
argentiana_crs_list = search_crs_by_area(
    area_name=argentina_aoi,
    object_type="GEOGRAPHIC_2D_CRS",
    filter_text="Argentina"
)

In [42]:
for crs in argentiana_crs_list:
    print(f"EPSG:{crs.code} - {crs.name} - Type: {crs.type}")
    #print(f"Area of Use: {crs.area_of_use}")

EPSG:4160 - Chos Malal 1914 - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:4161 - Pampa del Castillo - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:4190 - POSGAR 98 - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:4221 - Campo Inchauspe - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:4254 - Hito XVIII 1963 - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:4326 - WGS 84 - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:4694 - POSGAR 94 - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:5340 - POSGAR 2007 - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:9248 - Tapi Aike - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:9251 - MMN - Type: PJType.GEOGRAPHIC_2D_CRS
EPSG:9253 - MMS - Type: PJType.GEOGRAPHIC_2D_CRS
